<a href="https://colab.research.google.com/github/cass-android/international_energy_stats/blob/master/International_Energy_Stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [229]:
import pandas as pd
import re
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Notes on Data 

**Sources**

*   http://data.un.org/Explorer.aspx


**Units and number of categories using them:**

*   'Cubic metres, thousand' : 1 (fuelwood)
*   'Metric tons,  thousand' : 48 
*   'Kilowatt-hours, million' :  9
*   'Terajoules' : 15
*   'kW' : 1 (electricity_net_installed_capacity_of_electric_power_plants')
*   'Metric Tons' : 1 uranium (reserves and production)

    
 **Resources used for conversion factors:**

*  https://millenniumindicators.un.org/unsd/energy/balance/2013/05.pd
*   https://www150.statcan.gc.ca/n1/pub/57-601-x/2010004/appendix-appendice1-eng.htm
*   https://unstats.un.org/unsd/energy/balance/conversion.htm (2004)


**Other notes:**

*   Data does not appear to include electricity end uses
*    'Final energy consumption' Refers to all fuel and energy that are delivered to users for their energy use. Includes transportation.
*    'Production' Seems to include both primary and secondary (e.g. electricity)


   
   




In [0]:
data = pd.read_csv('/content/drive/My Drive/Data/all_energy_statistics_2019.csv')
cf = pd.read_csv('/content/drive/My Drive/Data/conversionfactors.csv')

# Convert Quantity to TJ

data = data.merge(cf, how='left', on='category')

data.insert( 
    column='Quantity_TJ', 
    loc=8, 
    value= data['Quantity'] * 
    data['TJ_conversion']
)


data.insert(column='Flow_Category', loc=7, value="")
data.drop('Unit_y', axis=1, inplace=True)
data['Year'] = data['Year'].fillna(0).astype(int)


In [0]:
# Add Per-capita from population stats

pop = pd.read_csv('/content/drive/My Drive/Data/population.csv', header = 4).drop(
    columns=['Country Code','Indicator Name','Indicator Code']
)

pop = pd.DataFrame(pop.set_index('Country Name').unstack())
data['Year'] = data['Year'].astype(str)
data = data.merge(pop, how='left', left_on=['Year','Country or Area'], right_index=True)
data = data.rename({0:'Population'}, axis=1)
data.insert(
            column = 'TJ_per_capita',
            loc = 11,
            value = data['Quantity_TJ'].divide(data['Population'])
            )

In [0]:
# Add Per-GDP from gdp stats

gdp = pd.read_csv('/content/drive/My Drive/Data/gdp_usd.csv', header = 4).drop(
    columns=['Country Code','Indicator Name','Indicator Code']
)
gdp = pd.DataFrame(gdp.set_index('Country Name').unstack())
data['Year'] = data['Year'].astype(str)
data = data.merge(gdp, how='left', left_on=['Year','Country or Area'], right_index=True)
data = data.rename({0:'GDP_USD'}, axis=1)
data.insert(
            column = 'TJ_per_GDP$',
            loc = 11,
            value = data['Quantity_TJ'].divide(data['GDP_USD'])
            )

In [0]:
# Add 'Flow_Category' (from Commodity - Transaction)

Flow_Categories = [
    'Receipts from other sources', 
    'Imports',
    'Exports',
    'International marine bunkers',
    'International aviation bunkers',
    'Stock changes',
    'Transfers',
    'Transformation',
    'Energy industries own use',
    'Losses',
    'Final consumption',
    'Non-energy uses',
    'Final energy consumption', 
    'Production'
]

def function(row) :
    for i in Flow_Categories:
        if re.search(i.lower(), row['Commodity - Transaction'].lower()):
            row['Flow_Category'] = i
            return row
    return row
  
data = data.apply(
    func=function,
    axis=1
)

In [276]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1320099 entries, 0 to 1320098
Data columns (total 14 columns):
Country or Area            1320099 non-null object
Commodity - Transaction    1320099 non-null object
Year                       1320099 non-null object
Unit_x                     1319949 non-null object
Quantity                   1319949 non-null float64
Quantity Footnotes         178084 non-null float64
category                   1320099 non-null object
Flow_Category              1320099 non-null object
Quantity_TJ                1257990 non-null float64
TJ_conversion              1258136 non-null float64
Population                 1119610 non-null float64
TJ_per_GDP$                1044557 non-null float64
TJ_per_capita              1068649 non-null float64
GDP_USD                    1093640 non-null float64
dtypes: float64(8), object(6)
memory usage: 151.1+ MB


In [0]:
data.to_csv('/content/drive/My Drive/Data/all_energy_statistics_2019_V2.csv')

In [0]:
# for later...?

totals = data[data['Year']==1990].groupby(by=['Country or Area'])['Quantity'].sum()

# Sankey : for another day

import urllib
import json
url = 'https://raw.githubusercontent.com/plotly/plotly.js/master/test/image/mocks/sankey_energy.json'
response = urllib.request.urlopen(url)
sankey = json.loads(response.read())